In [173]:
import numpy as np
import pandas as pd
from astropy import units as u
import matplotlib.pyplot as plt
from astropy.cosmology import FlatLambdaCDM

# Load the data
# df = pd.read_csv('../DATA/ZFOURGE/CDFS/CDFS_MAIN7.csv', index_col=['Seq'])
# df = pd.read_csv('../DATA/ZFOURGE/COSMOS/COSMOS_MAIN7.csv', index_col=['Seq'])
df = pd.read_csv('../DATA/ZFOURGE/UDS/UDS_MAIN7.csv', index_col=['Seq'])
# df = df[(df['Use'] == 1) & (df['LIR'] > 0)]

# Redshift bins
redshift_bins = [(0.00, 0.30), (0.30, 0.45), (0.45, 0.60),
                 (0.60, 0.80), (0.80, 1.00), (1.00, 1.20),
                 (1.20, 1.70), (1.70, 2.00), (2.00, 2.50),
                 (2.50, 3.00), (3.00, 4.20), (4.20, 6.00)]

# Redshift error per redshift bin
redshift_bin_error = [0.027, 0.032, 0.036, 
                      0.030, 0.039, 0.042, 
                      0.061, 0.070, 0.087, 
                      0.106, 0.097, 0.123]

# Luminosity error per redshift bin
luminosity_perc_bin_error = [18.14, 23.45, 29.13, 
                             21.35, 32.03, 19.88, 
                             25.21, 18.88, 20.48, 
                             26.84, 18.33, 20.47] # percentage

In [174]:
# Luminosity
luminosity = df['LIR'].copy() # solar luminosities
luminosity *= 3.828 * 10 ** 26 # solar luminosity -> Watts

# Luminosity distance
z = df['zpk'].values # redshift
cosmo = FlatLambdaCDM(H0=70, Om0=0.3) # cosmology
d_L = cosmo.luminosity_distance(z).to(u.m).value # meters

# Bolometric flux
F_bol = luminosity / (4 * np.pi * (d_L ** 2)) # Watts / m^2

C:\Users\n10480129\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\astropy\cosmology\flrw\lambdacdm.py:404: RuntimeWarning: invalid value encountered in sqrt
  return 2 * np.sqrt(x) * hyp2f1(1.0 / 6, 1.0 / 2, 7.0 / 6, -(x**3))


In [175]:
all_L_up = []
all_L_down = []
all_indexs = []

# Calculate the luminosity errors
for (z_start, z_end), z_err, l_err in zip(redshift_bins, redshift_bin_error, luminosity_perc_bin_error):
    mask_z = (z >= z_start) & (z < z_end)
            
    # Perturb the redshift
    z_up = z[mask_z] + z_err
    z_down = z[mask_z] - z_err
    
    # Recalculate luminosity distance
    d_L_up = cosmo.luminosity_distance(z_up).to(u.m).value
    d_L_down = cosmo.luminosity_distance(z_down).to(u.m).value
    
    # Luminosity from redshift
    L_up = 4 * np.pi * (d_L_up ** 2) * F_bol[mask_z] * (2.612 * 10 ** -27) # Solar luminosities
    L_down = 4 * np.pi * (d_L_down ** 2) * F_bol[mask_z] * (2.612 * 10 ** -27) # Solar luminosities
    
    # Luminosity from error
    L_up *= (1+l_err/100)
    L_down *= (1-l_err/100)
    
    # Append to list
    all_L_up.append(L_up)
    all_L_down.append(L_down)
    all_indexs.append(L_up.index)

In [176]:
new_df = pd.DataFrame({
    'Seq': np.concatenate(all_indexs),
    'l_down': np.concatenate(all_L_down),
    'l_up': np.concatenate(all_L_up)
})

# Save to csv
new_df.sort_values(by='Seq', inplace=True)
new_df.to_csv('lum_error.csv', index=False, columns=['Seq', 'l_down', 'l_up'])

In [ ]:
colours = plt.cm.gist_rainbow(np.linspace(0,1,len(redshift_bins)))

for i, ((z_start, z_end), L_up, L_down, c, z_err) in enumerate(zip(redshift_bins, all_L_up, all_L_down, colours, redshift_bin_error)):
    mask_z = (z >= z_start) & (z < z_end)
    x = z[mask_z]
    y = np.log10(luminosity[mask_z] / (3.828 * 10 ** 26))
    
    if i == 1:
        plt.errorbar(x, y, yerr=[y - np.log10(L_down), np.log10(L_up) - y], fmt='o', color=c, capsize=4)

plt.xlabel('Redshift')
plt.ylabel('$\log(L_{IR})$ [$L_{\odot}$]')
plt.show()